# Compute simulation and subject bounds for Daw 2-step task

This following scripts compute the empirical information bottleneck, predictive information, and distance from the bound for all of the simulations and all of the subjects from the Kool et al., 2016, PLoS Comp. Biol. (https://doi.org/10.1371/journal.pcbi.1005090) assesment of model-based and model-free strategies in this task environment (their code and data can be found here: https://github.com/wkool/tradeoffs).

The results from these processing steps are used in 'Filipowicz_etal_EIB_MBMF.ipynb' for the main analyses and comparisons.

## 0) Loading necessary packages and helper functions

In [4]:
%pylab
%matplotlib inline
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import spearmanr, entropy
import scipy.io as spio

import glob, pickle

import sys, csv


######################
## HELPER FUNCTIONS ##
######################

#Functions to creat labels
def trial_label(x):
    as_strings = [num.astype(str) for num in x]
    return "".join(as_strings)

def make_features(trials_data):
    labeled_data = np.apply_along_axis(trial_label, 0, trials_data)
    combos = np.unique(labeled_data)
    string_to_index = dict(zip(combos, np.arange(len(combos))))
    map_to_index = np.vectorize(lambda x: string_to_index[x])
    mapped_data = map_to_index(labeled_data)
    return mapped_data

def get_marginal(x):
    """
    Helper function to compute and return marginal probability distribution for a 1d vector (x)
    """
    px = np.array([np.sum(x==xi) for xi in np.sort(np.unique(x))])/len(x)
    return(px)

def get_joint(x, y):
    """
    Computes joint probability distribution between 1d vectors x and y
    """
    #  set up dictionary for joint distribution (x-->y-->freq)
    joint_x_y = {}
    
    for x_un in np.unique(x):
        joint_x_y[x_un] = dict(zip(np.unique(y), np.zeros(len(np.unique(y)))))
        
#    populate dictionary 
    for trial, x_val in enumerate(x):
        y_val = y[trial]
        joint_x_y[x_val][y_val] += 1
        
#   normalize to make distirbution  
    joint_sum = sum(sum(list(c.values())) for c in list(joint_x_y.values()))
    
    for key1 in joint_x_y:
        for key2 in joint_x_y[key1]:
            joint_x_y[key1][key2] /= joint_sum
            
    return(joint_x_y)

def mutual_inf(x, y):
    """
    Calculates the mutual information I(x;y)
    Assuming x,y are both [n x 1] dimensional
    """  
#     Calculate marginal distributions
    px = get_marginal(x)
    py = get_marginal(y)
    
    
    joint_x_y = get_joint(x,y)
# calculate mutual information
    mi = 0
    
    for n_x, x_un in enumerate(np.unique(x)):
        pxi = px[n_x] # p(x)
        
        for n_y, y_un in enumerate(np.unique(y)):
            pyi = py[n_y] # p(y)            
            
            joint_i = joint_x_y[x_un][y_un] # P(x,y)
            
            if ((pxi == 0) or (pyi == 0) or (joint_i ==0 )):
                continue
            else:
                mi += joint_i * np.log2(joint_i/(pxi*pyi))
                
    return mi

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [10]:
# Compute/extract complexity, predictive accuracy, % correct, RT, main effect, and interaction for each subject
# Load all data - containes trial by trial responses/stimuli and also subject by subject interaction values and previous fits from Kool et al.
mbmf_subs = pd.read_csv('./data/MBMF_subject_data.csv')
subID = pd.unique(mbmf_subs['subject'])

# Get subject fits - here subject name is by index rather than ID but matches the order of the subject IDs in the file above
mbmf_fits = pd.read_csv('./data/w_fits.csv')

subInfo = {}
for i in np.arange(len(subID)):
    # Sub dictionary for each subject
    subInfo[subID[i]] = {}
    
    # Get subset of data with subject data
    sdat = mbmf_subs.loc[mbmf_subs['subject']==subID[i]]
    
    # Get trial info needed to compute complexity
    R1 = np.array(sdat['R1'].loc[sdat['R1'] >= 0]-1)
    S2 = np.array(sdat['S2'].loc[sdat['R1'] >= 0]-1)
    Rw = np.array(sdat['Rw'].loc[sdat['R1'] >= 0])
    R1_star = np.array(sdat['R1_star'].loc[sdat['R1'] >= 0]-1)
    
    # Get first-step RT info
    rt1 = np.array(sdat['rt1'].loc[sdat['R1'] >= 0])
    
    # Compute complexity for different feature sizes
    # All past features
    all_stack = np.stack((R1,S2,Rw,R1_star),axis=1)
    all_features = np.sum(all_stack*(2**np.arange(4)),axis=1)
    subInfo[subID[i]]['Ipast_all'] = mutual_inf(all_features[:-1],R1[1:])
    
    # Reduced features
    # No R1
    noR1_features = np.sum(all_stack[:,1:4]*(2**np.arange(3)),axis=1)
    subInfo[subID[i]]['Ipast_noR1'] = mutual_inf(noR1_features[:-1],R1[1:])
    
    # No S2
    noS2_features = np.sum(all_stack[:,[0,2,3]]*(2**np.arange(3)),axis=1)
    subInfo[subID[i]]['Ipast_noS2'] = mutual_inf(noS2_features[:-1],R1[1:])
    
    # No Rw
    noRw_features = np.sum(all_stack[:,[0,1,3]]*(2**np.arange(3)),axis=1)
    subInfo[subID[i]]['Ipast_noRw'] = mutual_inf(noRw_features[:-1],R1[1:])
    
    # No R1_star
    noR1_star_features = np.sum(all_stack[:,0:3]*(2**np.arange(3)),axis=1)
    subInfo[subID[i]]['Ipast_noR1_star'] = mutual_inf(noR1_star_features[:-1],R1[1:])
    
    # Predictive info and proportion correct
    # Ifuture
    subInfo[subID[i]]['Ifuture'] = mutual_inf(R1,R1_star)
    
    # Proportion correct
    subInfo[subID[i]]['PropCorr'] = np.mean(Rw)
    
    # Prop rich choices
    subInfo[subID[i]]['PropRich'] = np.mean(R1==R1_star)
    
    # Extract main effect reported by Kool et al
    subInfo[subID[i]]['ME'] = sdat['meffect'].iloc[0]
    
    # Extract interaction reported by Kool et al
    subInfo[subID[i]]['Int'] = sdat['interaction'].iloc[0]
    
    # Get mean log response times on first response
    subInfo[subID[i]]['Mean_logRT'] = np.mean(np.log(rt1[rt1>0]))

    # Get fit strategy mixing coefficient for stochastic model fits
    subInfo[subID[i]]['w_stoch'] = mbmf_fits['w_stoch'].iloc[i]
    
    # Get fit strategy mixing coefficient for deterministic model fits
    subInfo[subID[i]]['w_det'] = mbmf_fits['w_det'].iloc[i]

# Save individual subject info for use in the main analysis
outfile = open('./data/sub_cx_behav.pkl','wb')
pickle.dump(subInfo,outfile)
outfile.close()

/Users/alsfilip/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:57: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
